### 再現コード

In [26]:
import numpy as np
from scipy.optimize import root
from scipy.stats import genpareto
from scipy.optimize import fsolve


In [17]:
#Parameter Values

beta = 0.9747
h_M = 0.9
h_S = 1.4
alpha = -1.5
mu = 0.6
gamma = 1-mu #(needs to be 1-mu)
sigma = 0.6
N = 500
l_idx = np.arange(N)
a = 45 #Pareto coefficient
ksi1 = 1.142793
ksi2 = 0.0082433
delta = 7.5
Lbar = 100
kappa = 0.08
T = 240

In [19]:
# Arrays
w      = np.zeros((N,T), dtype=np.float64)
c_M    = np.zeros((N,T), dtype=np.float64)
c_S    = np.zeros((N,T), dtype=np.float64)
p_M    = np.zeros((N,T), dtype=np.float64)
p_S    = np.ones((N,T), dtype=np.float64)  # p_S は常に1
M  = np.zeros((N,T), dtype=np.float64)
S  = np.zeros((N,T), dtype=np.float64)
L_M    = np.zeros((N,T), dtype=np.float64)
L_S    = np.zeros((N,T), dtype=np.float64)
L      = np.zeros((N,T), dtype=np.float64)
R_M    = np.zeros((N,T), dtype=np.float64)
R_S    = np.zeros((N,T), dtype=np.float64)
phi_M  = np.zeros((N,T), dtype=np.float64)
phi_S  = np.zeros((N,T), dtype=np.float64)
theta  = np.zeros((N,T), dtype=np.int8)      # ← ここは int8/bool で節約
ES_M   = np.zeros((N,T), dtype=np.float64)

Ubar   = np.zeros(T, dtype=np.float64)
RI     = np.zeros(T, dtype=np.float64)
pM_first = np.zeros(T, dtype=np.float64)      # p_M の先頭地点

# “朝の生産性”は t+1 を使うので +1 期ぶん確保
Z_M_mor = np.zeros((N,T+1), dtype=np.float64)
Z_S_mor = np.zeros((N,T+1), dtype=np.float64)
Z_M     = np.zeros((N,T), dtype=np.float64)
Z_S     = np.zeros((N,T), dtype=np.float64)


In [ ]:
def residuals(X,t):
    # これは、X = [Ubar(t), p_M(1,t), RI(t)]を受け取って、各変数を計算する関数
    Ubar[t]   = X[0]
    p_M[0, t] = X[1]         # 先頭地点の価格
    RI[t]     = X[2]
    
    
    for i in range(N):
        # 比較生産性の計算
    
        x =  ((p_M[i,t]/h_M)/(p_S[i,t]/h_S)) ** (1/(alpha-1))
        w[i,t] = Ubar[t] * (x * p_M[i,t] + p_S[i,t]) / ((h_M * (x**alpha)+h_S)**(1/alpha)) - RI[t]
        c_S[i,t] = (w[i,t]+RI[t]) / (p_S[i,t]+p_M[i,t]*x)
        c_M[i,t] = c_S[i,t] * x
        
        phi_M[i,t] = 1 - ((Z_M[i,t] * p_M[i,t] * gamma)/(ksi2*(a-1)*(w[i,t]/(mu*p_M[i,t]))**((1-gamma)/gamma))) **(-1/2)
        
        phi_S[i,t] = 1 - ((Z_S[i,t] * p_S[i,t] * gamma)/(ksi2*(a-1)*(w[i,t]/(sigma*p_S[i,t]))**((1-gamma)/gamma))) **(-1/2)
        
        if phi_M[i,t] < 0:
            phi_M[i,t] = 0
        if phi_M[i,t] > 1:
            phi_M[i,t] = 1
        if phi_S[i,t] < 0:
            phi_S[i,t] = 0
        if phi_S[i,t] > 1:
            phi_S[i,t] = 1

        L_M[i,t] = ((w[i,t])/(mu*p_M[i,t] * (Z_M[i,t] * (phi_M[i,t]/(a-1) + 1))**gamma)) ** (1/(mu-1))
        L_S[i,t] = ((w[i,t])/(sigma*p_S[i,t] * (Z_S[i,t] * (phi_S[i,t]/(a-1) + 1))**gamma)) ** (1/(sigma-1))
        
        if (ksi1 + ksi2/(1-phi_M[i,t])) * w[i,t] > ( (Z_M[i,t]**gamma * p_M[i,t] * L_M[i,t]**mu)/((a-1)**gamma) ) * ((phi_M[i,t]+a-1)**gamma - (a-1)**gamma) :
            phi_M[i,t] = 0
            L_M[i,t] = ((w[i,t])/(mu*p_M[i,t] * (Z_M[i,t])**gamma)) ** (1/(mu-1))
        if (ksi1 + ksi2/(1-phi_S[i,t])) * w[i,t] > ( (Z_S[i,t]**gamma * p_S[i,t] * L_S[i,t]**sigma)/((a-1)**gamma) ) * ((phi_S[i,t]+a-1)**gamma - (a-1)**gamma) :
            phi_S[i,t] = 0
            L_S[i,t] = ((w[i,t])/(sigma*p_S[i,t] * (Z_S[i,t])**gamma)) ** (1/(sigma-1))
            
        R_M[i,t] = p_M[i,t] * (Z_M[i,t] * (phi_M[i,t]/(a-1) + 1))**gamma * L_M[i,t]**mu - w[i,t] * L_M[i,t] - (ksi1 + ksi2/(1-phi_M[i,t])) * w[i,t]
        R_S[i,t] = p_S[i,t] * (Z_S[i,t] * (phi_S[i,t]/(a-1) + 1))**gamma * L_S[i,t]**sigma - w[i,t] * L_S[i,t] - (ksi1 + ksi2/(1-phi_S[i,t])) * w[i,t]
        
        if R_M[i,t] > R_S[i,t]:
            theta[i,t] = 1
            phi_S[i,t] = 0
        else:
            theta[i,t] = 0
            phi_M[i,t] = 0
        
        M[i,t] = (Z_M[i,t] * (phi_M[i,t]/(a-1) + 1))**gamma * L_M[i,t]**mu
        S[i,t] = (Z_S[i,t] * (phi_S[i,t]/(a-1) + 1))**gamma * L_S[i,t]**sigma
        
        L[i,t] = theta[i,t] * L_M[i,t] + (1-theta[i,t]) * L_S[i,t]
        
        if i == 0:
            if phi_M[i,t] > 0:
                ES_M[i,t] = (theta[i,t](M[i,t]-(w[i,t]/p_M[i,t]) * (ksi1 + ksi2/(1-phi_M[i,t]))) - c_M[i,t] * L_M[i,t]) / N
            else:
                ES_M[i,t] = (theta[i,t]*M[i,t] - c_M[i,t]*L_M[i,t]) / N
        if i > 0:
            if phi_M[i,t] > 0:
                ES_M[i,t] = ES_M[i-1,t] + (theta[i,t]*(M[i,t]-(w[i,t]/p_M[i,t]) * (ksi1 + ksi2/(1-phi_M[i,t]))) - c_M[i,t] * L_M[i,t] - kappa * np.abs(ES_M[i-1,t])) / N 
            else:
                ES_M[i,t] = ES_M[i-1,t] + (theta[i,t]*M[i,t] - c_M[i,t]*L_M[i,t] - kappa * np.abs(ES_M[i-1,t])) / N

        if ES_M[i,t] > 0 :
            if i < N :
                p_M[i+1,t] = p_M[i,t] * np.exp(kappa/N)
        else : 
            if i < N :
                p_M[i+1,t] = p_M[i,t] * np.exp(-kappa/N)       

    if (Ubar[t] > 0):# and sum((1-theta[:,t]) * L[:,t]) / N < Lbar :
        res1 = ES_M[N,t]
    else:
        res1 = 1000000000000

    if p_M[1,t] > 0:
        res2 = sum(L[:,t]) / N - Lbar
    else:
        res2 = 1000000000000
    
    if RI[t] > 0:
        res3 = RI[t] - (sum(theta[:,t]*R_M[:,t]) + sum((1-theta[:,t]) * R_S[:,t])) / (Lbar * N)
    else:
        res3 = 1000000000000
    
    return [res1, res2, res3]


In [ ]:
for t in range(T):
    if t == 0:
        Z_M_mor[:,0] = 0.8 + 0.4*(l_idx/N)
        Z_S_mor[:,0] = 1.0
    X0 = np.array([Ubar[t-1], p_M[0,t-1], RI[t-1]]) if t>0 else np.array([0.0492, 1.035, 0.054])
    X_star = fsolve(lambda X: residuals(X, t), X0)


In [ ]:
def diffusion_max_kernel(Z_t, delta):
    N = Z_t.shape[0]
    # 素直に O(N^2) でOK（N=500なら許容）。必要なら近傍半径で高速化
    Z_next = np.empty_like(Z_t)
    for i in range(N):
        weights = np.exp(-delta * np.abs(np.arange(N) - i) / N)
        Z_next[i] = np.max(weights * Z_t)
    return Z_next

In [ ]:
X0 = np.array([0.0492, 1.035, 0.054])  # 初期値 (np.array, shape (3,))


for t in range(T):

    sol = root(SDsysFinal, X0, method="hybr", tol=1e-4, options={"maxfev": 1000000})

    X = sol.x       # 解ベクトル (np.array, shape (3,))
    Eval = sol.fun  # 残差ベクトル (np.array, shape (3,))
    
    Ubar[t] = X[0]
    p_M[1,t] = X[1]
    RI[t] = X[2]
    
 

    # Diffusion
    Z_M_Mor[:, t+1] = diffusion_max_kernel(Z_M[:, t], delta)
    Z_S_Mor[:, t+1] = diffusion_max_kernel(Z_S[:, t], delta)
    


In [13]:
a = np.zeros((3,2))
print(a)
# aの2行1列目の要素を表示
print(a[1,0])

[[0. 0.]
 [0. 0.]
 [0. 0.]]
0.0


In [16]:
5^2

7